# Building Optimization with Dask

This notebook uses Dask to parrallize the NSGAII alogrithm.
Most of this notebook is copied from the "BuildingOptimization" notebook. But, this notebook will skip over concepts unrelated to Dask. Go to examples/Optimization/BuildingOptimization.ipynb for more details about the notebook.

### Setup dask
To use Dask setup the scheduler and the workers by runnning the cell below.
The client object is used to get and set various dask settings such as the number of workers.

If you're running this notebook locally, you should be able to open the dashboard using the link provided by client.

In [ ]:
import pandas as pd
from besos import eppy_funcs as ef
from besos.evaluator import EvaluatorEP
from besos.optimizer import NSGAII
from besos.parameters import RangeParameter, expand_plist, wwr
from besos.problem import EPProblem
from matplotlib import pyplot as plt

from dask.distributed import Client
client = Client()
client

### Building Optimization setup

In [ ]:
building = ef.get_building("in.idf")  # Load the E+ model in.idf

In [ ]:
parameters = []
parameters = expand_plist(  # Use helper function to make parameter list
    {
        "Building 1": {"North Axis": (0, 359)}  # Name from IDF Building object
    }  # Change orientation from North
)

parameters.append(
    wwr(RangeParameter(0.1, 0.9))
)  # Add window-to-wall ratio as a parameter between 0.1 and 0.9 using a custom function

In [ ]:
objectives = [
    "DistrictCooling:Facility",
    "DistrictHeating:Facility",
]  # Use Heating and Cooling (Ideal air loads) as objectives
problem = EPProblem(
    parameters, objectives
)  # Make a problem instance from the parameters and objectives

### Set up EnergyPlus evaluator
The Energy Plus evaluator must have `multi = True` to enable multiprocessing.
This disables the caching functionality of the evaluator which is incompatible with multiprocessing.
If a distributed network is used, `distributed = True` must also be set in the evaluator.
The `distributed` setting ensures that the eplus configuration files (.idf, .idd, and .epw) are sent to the workers.

In this cell, a evaluator is created and one simulation is run as a test

In [ ]:
evaluatorEP = EvaluatorEP(
    problem, building, multi=True
)  # outputdir must exist; E+ files will be written there
runs = pd.DataFrame.from_dict(
    {"0": [180, 0.5]}, orient="index"
)  # Make a dataframe of runs with one entry for South and 50% glazing
outputs = evaluatorEP.df_apply(runs)  # Run this as a test
outputs

### Run the Genetic Algorithm
Run the genetic algorith. Because the evaluator was created with `multi=True` the algorithm will also use multiprocessing
For each iteration of the algorithm, the evaluator will be run in parrallel for each worker available (indicated by the Dask client objct).

You might also see a bunch of warnings along the lines of "..full garbage collections...". I'm not sure exactly what's causing them, but restarting the workers from time to time will help minimize the warnings ~ goh.

In [ ]:
%%time
results = NSGAII(
    evaluatorEP, evaluations=20, population_size=50
)  # Run the optimizer using this evaluator for a population size of 20 for 10 generations
results

Compare with sequential processing. The performance difference will be greater if your machine has more cores.

In [ ]:
%%time
evaluatorEP = EvaluatorEP(problem, building)

results2 = NSGAII(
    evaluatorEP, evaluations=20, population_size=50
)  # Run the optimizer using this evaluator for a population size of 20 for 10 generations
results2

In [ ]:
optres = results.loc[
    results["pareto-optimal"] == True, :
]  # Get only the optimal results
plt.plot(
    results["DistrictCooling:Facility"], results["DistrictHeating:Facility"], "x"
)  # Plot all results in the background
plt.plot(
    optres["DistrictCooling:Facility"], optres["DistrictHeating:Facility"], "ro"
)  # Plot optimal results in red
plt.xlabel("Cooling demand")
plt.ylabel("Heating demand")

## Visualize the results

In [ ]:
optres = optres.sort_values("DistrictCooling:Facility")  # Sort by the first objective
optresplot = optres.drop(columns="violation")  # Remove the constraint violation column
ax = optresplot.plot.bar(
    subplots=True, legend=None, figsize=(10, 10)
)  # Plot the variable values of each of the optimal solutions

In [ ]:
client.restart()